
# 📊 Historical Backtest Engine: Mathematical Framework & Methodology

## 🎯 **Core Philosophy & Approach**

The Historical Backtest Engine operates on the principle of **realistic simulation** - every calculation mirrors real-world margin trading mechanics with institutional-grade precision. Unlike simplified backtests that ignore borrowing costs and margin requirements, this engine models the complete lifecycle of a leveraged position with daily granularity.

## 🏗️ **Data Architecture & Flow**

### **Primary Data Sources**
The engine uses your `ETFs and Fed Funds Data.xlsx` as the authoritative source, containing:
- **Daily ETF Prices**: SPY and VTI closing prices
- **Dividend Payments**: Quarterly dividend distributions  
- **Fed Funds Rates**: Historical Federal Reserve rates for realistic interest calculations

### **Data Processing Logic**
```
Data Flow: Excel → Pandas DataFrame → Date-Indexed Time Series → Daily Simulation Loop
```

The system converts all data to datetime-indexed pandas DataFrames, enabling efficient vectorized operations and precise date-based filtering for any user-specified backtest period.

## ⚖️ **Margin Account Mechanics**

### **Account Type Specifications**

**Reg-T Account (Regulation T):**
- **Maximum Leverage**: 2:1 (50% initial margin requirement)
- **Maintenance Margin**: 25% of position value
- **Interest Rate**: Fed Funds Rate + 1.5% spread

**Portfolio Margin Account:**
- **Maximum Leverage**: 7:1 (dynamic initial margin requirement)
- **Maintenance Margin**: 15% of position value  
- **Interest Rate**: Fed Funds Rate + 2.0% spread (higher due to increased risk)

### **Initial Position Setup**

When a user specifies an investment amount and leverage, the system calculates:

$$\text{Cash Investment} = \frac{\text{Total Position Size}}{\text{Leverage Ratio}}$$

$$\text{Margin Loan} = \text{Total Position Size} - \text{Cash Investment}$$

$$\text{Initial Shares} = \frac{\text{Total Position Size}}{\text{Initial ETF Price}}$$

**Example**: $10M position with 4x leverage
- Cash Investment = $10M ÷ 4 = $2.5M
- Margin Loan = $10M - $2.5M = $7.5M  
- Shares (if SPY = $400) = $10M ÷ $400 = 25,000 shares

## 💰 **Interest Rate Calculations**

### **Dynamic Interest Rate Model**

The system uses **real historical Fed Funds rates** rather than static assumptions:

$$\text{Daily Margin Rate} = \frac{\text{Fed Funds Rate} + \text{Account Spread}}{365}$$

$$\text{Daily Interest Cost} = \text{Outstanding Margin Loan} \times \text{Daily Margin Rate}$$

### **Compounding Effect**

Interest compounds daily, meaning the margin loan grows continuously:

$$\text{New Margin Loan} = \text{Previous Margin Loan} + \text{Daily Interest Cost}$$

This creates a realistic cost structure where longer holding periods and higher leverage dramatically impact returns through cumulative interest expenses.

## 📈 **Dividend Reinvestment Logic**

### **Automatic Reinvestment Process**

When dividends are paid (typically quarterly), the system:

1. **Calculates Dividend Income**:
   $$\text{Dividend Received} = \text{Current Shares} \times \text{Dividend Per Share}$$

2. **Reinvests Immediately**:
   $$\text{Additional Shares} = \frac{\text{Dividend Received}}{\text{Current ETF Price}}$$

3. **Updates Share Count**:
   $$\text{New Total Shares} = \text{Previous Shares} + \text{Additional Shares}$$

This compounding effect means the position grows over time, increasing both potential gains and margin requirements.

## 🔍 **Daily Portfolio Valuation**

### **Core Portfolio Metrics**

Each trading day, the system recalculates:

$$\text{Portfolio Value} = \text{Current Shares} \times \text{Current ETF Price}$$

$$\text{Equity} = \text{Portfolio Value} - \text{Outstanding Margin Loan}$$

$$\text{Leverage Ratio} = \frac{\text{Portfolio Value}}{\text{Equity}}$$

### **Maintenance Margin Requirements**

$$\text{Required Maintenance Margin} = \text{Portfolio Value} \times \text{Maintenance \%}$$

Where Maintenance % = 25% for Reg-T, 15% for Portfolio Margin

## 🚨 **Margin Call Detection**

### **Margin Call Trigger Condition**

A margin call occurs when:

$$\text{Current Equity} < \text{Required Maintenance Margin}$$

### **Margin Call Price Calculation**

The system calculates the ETF price at which a margin call would trigger:

$$\text{Margin Call Price} = \frac{\text{Outstanding Margin Loan}}{\text{Current Shares} \times (1 - \text{Maintenance \%})}$$

This forward-looking calculation helps assess risk before it materializes.

## 📊 **Performance Metrics Calculations**

### **Total Return**

$$\text{Total Return \%} = \frac{\text{Final Equity} - \text{Initial Cash Investment}}{\text{Initial Cash Investment}} \times 100$$

### **Compound Annual Growth Rate (CAGR)**

$$\text{CAGR \%} = \left(\frac{\text{Final Equity}}{\text{Initial Cash Investment}}\right)^{\frac{1}{\text{Years}}} - 1 \times 100$$

Where Years = Trading Days ÷ 252 (approximate trading days per year)

### **Maximum Drawdown**

$$\text{Drawdown}_t = \frac{\text{Equity}_t - \text{Peak Equity}_t}{\text{Peak Equity}_t} \times 100$$

$$\text{Maximum Drawdown} = \min(\text{All Drawdowns})$$

Peak Equity is the rolling maximum equity value up to time t.

### **Annual Volatility**

$$\text{Daily Returns}_t = \frac{\text{Equity}_t - \text{Equity}_{t-1}}{\text{Equity}_{t-1}}$$

$$\text{Annual Volatility} = \text{std}(\text{Daily Returns}) \times \sqrt{252} \times 100$$

### **Sharpe Ratio**

$$\text{Sharpe Ratio} = \frac{\text{CAGR}}{\text{Annual Volatility}}$$

(Assuming zero risk-free rate for simplicity)

## 🔄 **Daily Simulation Loop Logic**

### **Sequential Processing**

For each trading day, the engine executes this sequence:

1. **Load Market Data**: Get ETF price, dividend payment, Fed Funds rate
2. **Calculate Interest**: Apply daily interest to outstanding margin loan
3. **Process Dividends**: If dividend paid, reinvest automatically
4. **Update Portfolio**: Recalculate portfolio value and equity
5. **Check Margin**: Determine if margin call triggered
6. **Store Results**: Save all metrics for analysis and visualization

### **State Persistence**

Critical state variables carry forward between days:
- **Outstanding Margin Loan** (growing with interest)
- **Total Shares Owned** (increasing with dividend reinvestment)
- **Cumulative Interest Paid**
- **Margin Call History**

## 📈 **Risk Assessment Framework**

### **Multi-Dimensional Risk Analysis**

The engine evaluates risk across several dimensions:

1. **Leverage Risk**: Higher leverage amplifies both gains and losses
2. **Interest Rate Risk**: Rising rates increase borrowing costs
3. **Market Risk**: ETF price volatility affects margin requirements
4. **Liquidity Risk**: Margin calls may force disadvantageous timing

### **Risk Indicators**

**Price Drop Tolerance**:
$$\text{Max Price Drop \%} = \left(1 - \frac{\text{Margin Call Price}}{\text{Current Price}}\right) \times 100$$

**Interest Coverage Ratio**:
$$\text{Coverage Ratio} = \frac{\text{Annual Dividend Income}}{\text{Annual Interest Cost}}$$

## 🎯 **Scenario Analysis Capabilities**

### **Historical Stress Testing**

The engine can test strategies through actual market crises:
- **2008 Financial Crisis**: Extreme volatility and drawdowns
- **2020 COVID Crash**: Rapid market decline and recovery  
- **2022 Rate Hikes**: Rising interest rate environment

### **Leverage Impact Analysis**

By comparing identical strategies with different leverage ratios, users can see the non-linear relationship between leverage and risk:

$$\text{Risk Multiplier} \neq \text{Leverage Multiplier}$$

Higher leverage creates disproportionate risk due to:
- Compounding interest costs
- Increased margin call probability
- Reduced flexibility during market stress

## 🔬 **Accuracy & Validation**

### **Realistic Assumptions**

The engine incorporates realistic constraints:
- **No Slippage**: Assumes perfect execution (slight optimistic bias)
- **Infinite Liquidity**: No market impact from position size
- **Immediate Reinvestment**: Dividends reinvested at closing price
- **Daily Interest**: Interest calculated and compounded daily

### **Data Integrity**

All calculations use **actual historical data**:
- Real ETF prices from 1993/2001 onwards
- Actual dividend payment dates and amounts
- Historical Federal Reserve interest rates
- No synthetic or modeled data

## 🎨 **Visualization Mathematics**

### **Multi-Panel Chart Structure**

**Panel 1 - Portfolio Evolution**: 
- Portfolio Value vs. Equity over time
- Margin call events marked as red X symbols

**Panel 2 - Drawdown Analysis**:
- Running drawdown percentage from peak equity
- Filled area showing underwater periods

**Panel 3 - Interest Rate Environment**:
- Fed Funds rate vs. actual margin rate paid
- Visual correlation between rate changes and performance

### **Interactive Analytics**

Each chart point contains metadata:
- Exact date and values
- Margin loan outstanding
- Shares owned
- Interest rate environment

## 🧠 **Strategic Insights Generated**

### **Key Learnings**

The backtesting reveals critical insights:

1. **Interest Rate Sensitivity**: How rate changes affect leveraged returns
2. **Dividend Impact**: Quantifies the compounding benefit of reinvestment  
3. **Crisis Performance**: How strategies perform during market stress
4. **Optimal Leverage**: Risk-adjusted leverage ratios for different market conditions
5. **Timing Effects**: How entry timing affects long-term outcomes

### **Decision Framework**

Users can evaluate:
- **Risk Tolerance**: Maximum acceptable drawdown levels
- **Return Expectations**: Realistic performance targets
- **Time Horizon**: How holding period affects outcomes
- **Market Conditions**: When leverage enhances vs. hurts performance

---

## 💎 **The Mathematical Beauty**

This engine transforms raw market data into actionable intelligence through the elegant interplay of:

- **Compound Growth**: Both beneficial (dividends) and detrimental (interest)
- **Risk-Return Dynamics**: Non-linear relationships revealed through simulation
- **Market Reality**: Real costs, real constraints, real outcomes
- **Temporal Precision**: Daily granularity capturing market microstructure

**The result**: A mathematically rigorous, empirically grounded tool that demystifies leveraged investing through the lens of historical reality.
